In [1]:
import datetime
from pyspark import sql
from lib import common_functions
from lib import configuration

In [2]:
spark = common_functions.get_spark_session('monster')
spark.active()

Variables

In [8]:
#export_name = 'employer_users'
export_name = 'bq-results'

old_path = configuration.monster_input_path+f'{export_name}-DL.csv'
new_path = configuration.monster_input_path+f'{export_name}-DWH.csv'

new_path

'/home/jovyan/code/files/input/monster/bq-results-DWH.csv'

Dataframes

In [9]:
df_old = spark.read.format("csv").options(header='true', inferSchema='true', delimiter=',').load(old_path)
df_new = spark.read.format("csv").options(header='true', inferSchema='true', delimiter=',').load(new_path)

In [14]:
df_old = df_old.filter(df_old.rowNoUpdatedTime.cast('int') == 1)
df_new = df_new.filter(df_new.rowNoUpdatedTime.cast('int') == 1)

In [15]:
print(df_old.count())
print(df_new.count())

print(df_old.count()-df_new.count())

78511
78428
83


In [16]:
missing_in_new = df_old.join(df_new, on=['ecomAccountId'], how='left_anti')
print(missing_in_new.count())
missing_in_new.show()

83
+--------------------+--------------------+--------------------+--------------------+--------------------------+----------------+
|       ecomAccountId|    billingAccountId|     createdByUserId|         accountName|employerAccountCreatedTime|rowNoUpdatedTime|
+--------------------+--------------------+--------------------+--------------------+--------------------------+----------------+
|42c89cfb-3a31-4b7...|8a129f7a946d3ee50...|caa8149c-4755-40c...|Stadtbäckerei Küh...|      2025-01-16 09:31:...|               1|
|89f1bc98-f121-440...|8a129b5d946998380...|c25bee86-5658-4be...|Tyler Athletic Fi...|      2025-01-16 03:13:...|               1|
|6f4f2bd3-a378-4f4...|8a128757946d53850...|8845f74b-8882-493...|    Chardam Gear co.|      2025-01-16 12:50:...|               1|
|0c44724b-3cee-4d8...|8a1289b9946d25410...|817f1ace-cafc-4dc...|Sketch Brahma Tec...|      2025-01-16 07:47:...|               1|
|8a0a8ccf-1514-41f...|8a12837e946d253d0...|07a93a88-38a4-4b9...|Pencil Perfection...|  

In [17]:
missing_in_new.toPandas().to_csv(f'{configuration.monster_output_path}{export_name}_prd_missing_DWH_vs_DL_{datetime.date.today()}.csv')

Fix Column names